## Metrics Evaluation
### Import Libraries

In [1]:
import numpy as np
import cv2 as cv
from PIL import Image, UnidentifiedImageError
import os

In [2]:
Image.MAX_IMAGE_PIXELS = None  # Override PIL's DecompressionBombError

## Load in noisy and final Images

In [67]:
BASE_DIR = os.path.join(os.getcwd(), 'output')

noisy = os.path.join(BASE_DIR, 'noisy')
final = os.path.join(BASE_DIR, 'final')
denoi = os.path.join(BASE_DIR, 'denoised')

In [68]:
os.listdir(noisy)

['19648211_HV.npy',
 '19648211_HV.tif',
 '19648321_RH.npy',
 '19648321_RH.tif',
 '196488211_HH.npy',
 '196488211_HH.tif',
 '196488311_RH.npy',
 '196488311_RH.tif',
 'README.md',
 'RH.npy',
 'RH.tif',
 'RV.npy',
 'RV.tif']

In [69]:
os.listdir(final)

['README.md']

In [70]:
os.listdir(denoi)

['19648211_HV.npy',
 '19648211_HV.tif',
 '19648321_RH.npy',
 '19648321_RH.tif',
 '196488211_HH.npy',
 '196488211_HH.tif',
 '196488311_RH.npy',
 '196488311_RH.tif',
 'README.md',
 'RH.npy',
 'RH.tif',
 'RV.npy',
 'RV.tif']

In [71]:
noisy = os.path.join(noisy, os.listdir(noisy)[1])
#final = os.path.join(final, os.listdir(final)[1])
denoi = os.path.join(denoi, os.listdir(denoi)[1])
final = denoi

In [72]:
img_n = np.asarray(Image.open(noisy).convert('L'))  # Noisy
img_f = np.asarray(Image.open(final).convert('L'))  # Filtered
img_d = np.asarray(Image.open(denoi).convert('L'))  # Denoised

### SSI Speckle Suppression Index

In [16]:
sigma_r = np.std(img_f)
r_dash = np.mean(img_f)

sigma_f = np.std(img_n)
f_dash = np.mean(img_n)

In [17]:
SSI = (sigma_r * f_dash) / (r_dash * sigma_f)
SSI

0.5310644307422371

In [19]:
## For Denoised Alone

sigma_r = np.std(img_d)
r_dash = np.mean(img_d)

sigma_f = np.std(img_n)
f_dash = np.mean(img_n)

SSI = (sigma_r * f_dash) / (r_dash * sigma_f)
SSI

0.5310644307422371

### ENL Equivalent Number of Looks

In [64]:
def ENL(filtered):
    """ Equivalent Number of Looks """
    mu = np.mean(filtered)
    sigma = np.std(filtered)

    ENL = (mu / sigma)**2
    print(f'        [*] ENL @ {ENL}')
    return ENL
ENL(img_f)

        [*] ENL @ 3.875379343701045


3.875379343701045

In [21]:
## For Denoised Alone

mu = np.mean(img_d)
sigma = np.var(img_d)

ENL = (mu / sigma)**2
ENL

0.014839786131379807

## SMPI Speckle Suppression and Mean Preservation Index

In [52]:
sigma_r = np.std(img_f)
r_dash = np.mean(img_f)

sigma_f = np.std(img_n)
f_dash = np.mean(img_n)

Q = 1 + abs(f_dash - r_dash)

SMPI = Q * (sigma_r / sigma_f)
SMPI

1.4624784582772128

In [23]:
## For Denoised Alone

sigma_r = np.std(img_d)
r_dash = np.mean(img_d)

sigma_f = np.std(img_n)
f_dash = np.mean(img_n)

Q = 1 + abs(f_dash - r_dash)

SMPI = Q * (sigma_r / sigma_f)
SMPI

2.595264426020416

## SSIM

In [24]:
from skimage.metrics import structural_similarity as ssim

max(img_f.max(), img_n.max())

255

In [ ]:
ssim = ssim(img_f, img_n, data_range=max(img_f.max(), img_n.max()) - min(img_f.min(), img_n.min()))

## PSNR

In [21]:
from skimage.metrics import peak_signal_noise_ratio as psnr

def PSNR(img_n, img_f):
    PSNR = psnr(img_f, img_n, data_range=max(img_f.max(), img_n.max()) - min(img_f.min(), img_n.min()))
    print(f'        [*] PSNR @ {PSNR}')
    return PSNR

In [30]:
PSNR(img_n, img_f)

TypeError: 'numpy.float64' object is not callable

## Cropping for ENL

In [73]:
img_n_raster = Image.open(noisy).convert('L')
img_f_raster = Image.open(final).convert('L')

In [74]:
width, height = img_n_raster.size 
  
# Setting the points for cropped image 
left = 8000
top = 8000
right = 10000
bottom = 10000

(left, top, right, bottom)

(8000, 8000, 10000, 10000)

In [75]:
n_crop = img_n_raster.crop((left, top, right, bottom)) 
f_crop = img_f_raster.crop((left, top, right, bottom)) 

In [76]:
f_crop.show() 

In [77]:
ENL(np.asarray(n_crop))

        [*] ENL @ 3.21249437361848


3.21249437361848

In [78]:
ENL(np.asarray(f_crop))

        [*] ENL @ 22.629790996528637


22.629790996528637